# **1) Import the Modules**

Modules are code libraries that contain a set of ready-to-use functions.

* The `ee` module allows developers to interact with Google Earth Engine using the Python programming language.
* The `os` module provides functions to perform tasks such as file and directory operations, process management, and environment variable manipulation.
* The `json` module allows developers to load, read and write JSON files.
* The `yaml` module allows developers to load, read and write YAML files.
* The `sys` module contains methods and variables for modifying many elements of the Python runtime environment.
* The `tabulate` module allows the user to display data in a table format.
* The `google.colab` module provides access to some of the unique features and functionality of Google Colab.

In [ ]:
import ee
import os
import json
import math
import yaml
import datetime
import tabulate

from google.colab import drive

# **2) Authentication Procedure**

This section provides instructions for setting up the Google Earth Engine Python API on Colab and for setting up Google Drive on Colab. These steps should be performed each time you start/restart/rollback a Colab session.

## **2.1) GEE**

The `ee.Authenticate` function authenticates access to the Google Earth Engine servers, while the `ee.Initialize` function initializes it. After executing the following cell, the user is prompted to grant Google Earth Engine access to their Google account.

**Note:** The Earth Engine API is installed by default in Google Colaboratory.

In [ ]:
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=-o_7Qxd5sqGOFBEaBJ1IuLliyssviIiu-fnxhQLVNSc&tc=p6wXQOt2JZFRf6p6mz5AArpVyajhOTmuLwOqxLw_uMo&cc=rLndFZsQFJpfNCSNmuoZN6LOO7-Af4TyYAqmmNGqXAE

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfJohXmoW7bF1QhSp51gxWTdVrEfrb5oAXRSh_h6Y57MRmsRahSwVuEiFNI

Successfully saved authorization token.


*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_doiqkQG3NJ1t8IS?source=API


## **2.2) GD**

The `drive.mount` function allows access to specific folders of Google Drive. Granting access to Google Drive allows code running in the notebook to modify files in Google Drive.

**Note:** When using the `Mount Drive` button in the file browser, no authentication codes are required for notebooks edited only by the current user.

In [ ]:
drive.mount("/content/gdrive")

Mounted at /content/gdrive


# **3) Functions**

Data Processing

In [ ]:
def power_to_db(raster):
  """
  Description:
    Converts pixel values from power scale to dB scale.

  Arguments:
    raster (ee.Image): The raster with pixel values in power scale.

  Returns:
    The raster with pixel values in dB scale.
  """
  return ee.Image(10).multiply(raster.log10())


def db_to_power(raster):
  """
  Description:
    Converts pixel values from dB scale to power scale.

  Args:
    raster (ee.Image): The raster with pixel values in dB scale.

  Returns:
    The raster with pixel values in power scale.
  """
  return ee.Image(10).pow(raster.divide(10))


def refined_lee(raster):
  """
  Description:
    Applies the refined Lee speckle filter to an raster.

  Arguments:
    raster (ee.Image): The raster to apply the filter on.

  Returns:
    The filtered raster layer.
  """
  def computations(b):
    img = raster.select([b])

    # img must be in natural units, i.e. not in dB!
    # Set up 3x3 kernels
    weights3 = ee.List.repeat(ee.List.repeat(1,3),3)
    kernel3 = ee.Kernel.fixed(3,3, weights3, 1, 1, False)

    mean3 = img.reduceNeighborhood(ee.Reducer.mean(), kernel3)
    variance3 = img.reduceNeighborhood(ee.Reducer.variance(), kernel3)

    # Use a sample of the 3x3 windows inside a 7x7 windows to determine gradients and directions
    sample_weights = ee.List([[0,0,0,0,0,0,0], [0,1,0,1,0,1,0],[0,0,0,0,0,0,0], [0,1,0,1,0,1,0], [0,0,0,0,0,0,0], [0,1,0,1,0,1,0],[0,0,0,0,0,0,0]])

    sample_kernel = ee.Kernel.fixed(7,7, sample_weights, 3,3, False)

    # Calculate mean and variance for the sampled windows and store as 9 bands
    sample_mean = mean3.neighborhoodToBands(sample_kernel)
    sample_var = variance3.neighborhoodToBands(sample_kernel)

    # Determine the 4 gradients for the sampled windows
    gradients = sample_mean.select(1).subtract(sample_mean.select(7)).abs()
    gradients = gradients.addBands(sample_mean.select(6).subtract(sample_mean.select(2)).abs())
    gradients = gradients.addBands(sample_mean.select(3).subtract(sample_mean.select(5)).abs())
    gradients = gradients.addBands(sample_mean.select(0).subtract(sample_mean.select(8)).abs())

    # And find the maximum gradient amongst gradient bands
    max_gradient = gradients.reduce(ee.Reducer.max())

    # Create a mask for band pixels that are the maximum gradient
    gradmask = gradients.eq(max_gradient)

    # duplicate gradmask bands: each gradient represents 2 directions
    gradmask = gradmask.addBands(gradmask)

    # Determine the 8 directions
    directions = sample_mean.select(1).subtract(sample_mean.select(4)).gt(sample_mean.select(4).subtract(sample_mean.select(7))).multiply(1)
    directions = directions.addBands(sample_mean.select(6).subtract(sample_mean.select(4)).gt(sample_mean.select(4).subtract(sample_mean.select(2))).multiply(2))
    directions = directions.addBands(sample_mean.select(3).subtract(sample_mean.select(4)).gt(sample_mean.select(4).subtract(sample_mean.select(5))).multiply(3))
    directions = directions.addBands(sample_mean.select(0).subtract(sample_mean.select(4)).gt(sample_mean.select(4).subtract(sample_mean.select(8))).multiply(4))
    # The next 4 are the not() of the previous 4
    directions = directions.addBands(directions.select(0).Not().multiply(5))
    directions = directions.addBands(directions.select(1).Not().multiply(6))
    directions = directions.addBands(directions.select(2).Not().multiply(7))
    directions = directions.addBands(directions.select(3).Not().multiply(8))

    # Mask all values that are not 1-8
    directions = directions.updateMask(gradmask)

    # "collapse" the stack into a singe band image (due to masking, each pixel has just one value (1-8) in it's directional band, and is otherwise masked)
    directions = directions.reduce(ee.Reducer.sum())

    sample_stats = sample_var.divide(sample_mean.multiply(sample_mean))

    # Calculate localNoiseVariance
    sigmaV = sample_stats.toArray().arraySort().arraySlice(0,0,5).arrayReduce(ee.Reducer.mean(), [0])

    # Set up the 7*7 kernels for directional statistics
    rect_weights = ee.List.repeat(ee.List.repeat(0,7),3).cat(ee.List.repeat(ee.List.repeat(1,7),4))

    diag_weights = ee.List([[1,0,0,0,0,0,0], [1,1,0,0,0,0,0], [1,1,1,0,0,0,0], [1,1,1,1,0,0,0], [1,1,1,1,1,0,0], [1,1,1,1,1,1,0], [1,1,1,1,1,1,1]])

    rect_kernel = ee.Kernel.fixed(7,7, rect_weights, 3, 3, False)
    diag_kernel = ee.Kernel.fixed(7,7, diag_weights, 3, 3, False)

    # Create stacks for mean and variance using the original kernels. Mask with relevant direction.
    dir_mean = img.reduceNeighborhood(ee.Reducer.mean(), rect_kernel).updateMask(directions.eq(1))
    dir_var = img.reduceNeighborhood(ee.Reducer.variance(), rect_kernel).updateMask(directions.eq(1))

    dir_mean = dir_mean.addBands(img.reduceNeighborhood(ee.Reducer.mean(), diag_kernel).updateMask(directions.eq(2)))
    dir_var = dir_var.addBands(img.reduceNeighborhood(ee.Reducer.variance(), diag_kernel).updateMask(directions.eq(2)))

    # and add the bands for rotated kernels
    for i in range(1, 4):
      dir_mean = dir_mean.addBands(img.reduceNeighborhood(ee.Reducer.mean(), rect_kernel.rotate(i)).updateMask(directions.eq(2*i+1)))
      dir_var = dir_var.addBands(img.reduceNeighborhood(ee.Reducer.variance(), rect_kernel.rotate(i)).updateMask(directions.eq(2*i+1)))
      dir_mean = dir_mean.addBands(img.reduceNeighborhood(ee.Reducer.mean(), diag_kernel.rotate(i)).updateMask(directions.eq(2*i+2)))
      dir_var = dir_var.addBands(img.reduceNeighborhood(ee.Reducer.variance(), diag_kernel.rotate(i)).updateMask(directions.eq(2*i+2)))

    # "collapse" the stack into a single band image (due to masking, each pixel has just one value in it's directional band, and is otherwise masked)
    dir_mean = dir_mean.reduce(ee.Reducer.sum())
    dir_var = dir_var.reduce(ee.Reducer.sum())

    # A finally generate the filtered value
    varX = dir_var.subtract(dir_mean.multiply(dir_mean).multiply(sigmaV)).divide(sigmaV.add(1.0))

    b = varX.divide(dir_var)

    return dir_mean.add(b.multiply(img.subtract(dir_mean))) \
      .arrayProject([0])                                    \
      .arrayFlatten([["sum"]])                              \
      .float()

  bandNames = raster.bandNames()
  raster = db_to_power(raster)

  result = ee.ImageCollection(bandNames.map(computations)).toBands().rename(bandNames)
  return power_to_db(ee.Image(result))


def slope_correction(raster, dem):
  """
  Description:
    Performs slope correction on the input raster using a digital elevation model (DEM).

  Arguments:
    raster (ee.Image): The raster to perform slope correction on.
    dem (ee.Image): The digital elevation model used for correction.

  Returns:
    The slope-corrected raster layer.
  """
  # Obtain the geometry of the image.
  imgGeom = raster.geometry()
  # Clip the extent of the dem layer.
  srtm = ee.Image(dem).clip(imgGeom)
  # Convert pixel values from dB scale to linear scale.
  sigma0Pow = db_to_power(raster)

  # 2.1.1 Radar geometry
  # Compute the mean aspect of the terrain over the region covered by the raster.
  theta_i = raster.select("angle")
  phi_i = ee.Terrain.aspect(theta_i)                                        \
    .reduceRegion(ee.Reducer.mean(), theta_i.get("system:footprint"), 1000) \
    .get("aspect")

  # 2.1.2 Terrain geometry
  # Calculate the slope and aspect of the terrain over the region covered by the raster.
  alpha_s = ee.Terrain.slope(srtm).select("slope")
  phi_s = ee.Terrain.aspect(srtm).select("aspect")

  # 2.1.3 Model geometry
  # reduce to 3 angle
  phi_r = ee.Image.constant(phi_i).subtract(phi_s)

  # Perform some mathematical conversions
  # (convert pixel values from degrees to radians).
  phi_rRad = phi_r.multiply(math.pi / 180)
  alpha_sRad = alpha_s.multiply(math.pi / 180)
  theta_iRad = theta_i.multiply(math.pi / 180)
  ninetyRad = ee.Image.constant(90).multiply(math.pi / 180)

  # slope steepness in range (eq. 2)
  alpha_r = (alpha_sRad.tan().multiply(phi_rRad.cos())).atan()

  # slope steepness in azimuth (eq 3)
  alpha_az = (alpha_sRad.tan().multiply(phi_rRad.sin())).atan()

  # local incidence angle (eq. 4)
  theta_lia = (alpha_az.cos().multiply((theta_iRad.subtract(alpha_r)).cos())).acos()
  theta_liaDeg = theta_lia.multiply(180 / math.pi)
  # 2.2
  # Gamma_nought_flat
  gamma0 = sigma0Pow.divide(theta_iRad.cos())
  gamma0dB = ee.Image.constant(10).multiply(gamma0.log10())
  ratio_1 = gamma0dB.select("VV").subtract(gamma0dB.select("VH"))

  # Volumetric Model
  nominator = (ninetyRad.subtract(theta_iRad).add(alpha_r)).tan()
  denominator = (ninetyRad.subtract(theta_iRad)).tan()
  volModel = (nominator.divide(denominator)).abs()

  # apply model
  gamma0_Volume = gamma0.divide(volModel)
  gamma0_VolumeDB = ee.Image.constant(10).multiply(gamma0_Volume.log10())

  # we add a layover/shadow maskto the original implmentation
  # layover, where slope > radar viewing angle
  alpha_rDeg = alpha_r.multiply(180 / math.pi)
  layover = alpha_rDeg.lt(theta_i)

  # shadow where LIA > 90
  shadow = theta_liaDeg.lt(85)

  # calculate the ratio for RGB vis
  ratio = gamma0_VolumeDB.select("VV").subtract(gamma0_VolumeDB.select("VH"))

  output = gamma0_VolumeDB.addBands(ratio).addBands(alpha_r).addBands(phi_s).addBands(theta_iRad) \
    .addBands(layover).addBands(shadow).addBands(gamma0dB).addBands(ratio_1)

  return raster.addBands(
    output.select(["VV", "VH"], ["VV", "VH"]),
    None,
    True
  )


def export_tasks_viewer(exportTasksIds, tableFormat: str = "plain"):
  """
  Description:
    Displays a table view which contains useful information about the provided export tasks.

  Notes:
    * Task_Id: The task identifier.
    * Task_State: One of READY, RUNNING, COMPLETED, FAILED, CANCELLED, UNSUBMITTED or UNKNOWN.
    * Task_Type: One of EXPORT_IMAGE, EXPORT_TILES, EXPORT_FEATURES, EXPORT_VIDEO.
    * Task_Attempt: Number of attempts.
    * Task_Description: A human-readable description of the task.
    * Queue_Time: The time that is taken while being in a queue.
    * Execution_Time: The time spent by the servers executing the task.
    * Completion_Time: SUm of queue and execution times.
    * Error_Message: Failure reason. Appears only if state is FAILED. May also include other fields.

  Arguments:
    exportTasksIdsList (list) (mandatory) A list of export task identifiers.
    tableFormat (str) (optional) The table format to use. Defaults to "plain".

  Returns:
    None, displays the export tasks table.
  """
  taskInfo = []
  tableHeaders = [
    "Task_Id", "Task_State", "Task_Type", "Task_Attempt", "Task_Description",
    "Queue_Time", "Execution_Time", "Completion_Time", "Error_Message"
  ]
  tableFormats = tabulate._table_formats.keys()

  if tableFormat not in tableFormats:
    raise ValueError(f"Invalid table format. Choose from: `{tableFormats}`.")

  # Populate taskInfo.
  for exportTaskId in exportTasksIds:

    taskState = ee.data.getTaskStatus(exportTaskId)[0]["state"]
    taskType = ee.data.getTaskStatus(exportTaskId)[0]["task_type"]
    taskDescription = ee.data.getTaskStatus(exportTaskId)[0]["description"]
    startTimestamp = datetime.datetime.fromtimestamp(ee.data.getTaskStatus(exportTaskId)[0]["start_timestamp_ms"]/1000.0)
    updateTimestamp = datetime.datetime.fromtimestamp(ee.data.getTaskStatus(exportTaskId)[0]["update_timestamp_ms"]/1000.0)
    creationTimestamp = datetime.datetime.fromtimestamp(ee.data.getTaskStatus(exportTaskId)[0]["creation_timestamp_ms"]/1000.0)

    queueTime = None
    taskAttempt = None
    executionTime = None
    completionTime = None

    if taskState not in ["READY", "RUNNING"]:
      queueTime = (startTimestamp - creationTimestamp).total_seconds()
      executionTime = (updateTimestamp - startTimestamp).total_seconds()

    if taskState == "COMPLETED":
      taskAttempt = ee.data.getTaskStatus(exportTaskId)[0]["attempt"]
      completionTime = (updateTimestamp - creationTimestamp).total_seconds()

    try:
      errorMessage = ee.data.getTaskStatus(exportTaskId)[0]["error_message"]
    except KeyError:
      errorMessage = None  # This just means that the export task has not failed.

    taskInfo.append([exportTaskId, taskState, taskType, taskAttempt, taskDescription, queueTime, executionTime, completionTime, errorMessage])

  # Table display.
  table = tabulate.tabulate(taskInfo, headers=tableHeaders, tablefmt=tableFormat)
  print(table)

# **4) Parameters**

In [ ]:
# `Sentinel-1 GRD`
b1 = "VV"
vh = "VH"

# `Digital Elevation Model`
demProvider = "USGS"

# EMS case of interest
caseCode = "emsr692"
caseArea = "magnesia"

# Projection of interest
projectionCRS = "EPSG:4326"
projectionScale = 10

# `Classification`
rasterIdentifier = "041299_04E8BD_E9C5_050224_060B99_D80F_slope"
preEventRasterIdentifier = "S1A_IW_GRDH_1SDV_20220103T162359_20220103T162424_041299_04E8BD_E9C5"
postEventRasterIdentifier = "S1A_IW_GRDH_1SDV_20230907T162412_20230907T162437_050224_060B99_D80F"

classifierIdentifier = "users/stamlazaros/hua/t-h-e-s-i-s/assets/classifiers/base"
classifierFeatures = [
  "VHVHD", "VHVHQ", "VVVHD", "VVVHQ", "VVVVD", "VVVVQ", "NDPID",
  "PRE_VV", "PRE_VH", "PRE_NDPI", "POST_VV", "POST_VH", "POST_NDPI"
]

# GEE paths
destinationFolder = "users/stamlazaros/hua/t-h-e-s-i-s/case_studies/emsr692/rasters/classified/2023_09_07"

# GD paths
# configFile = "/content/gdrive/MyDrive/t-h-e-s-i-s/configurations/case_studies.json"
configFile = "/content/gdrive/MyDrive/t-h-e-s-i-s/configurations/case_studies.yaml"

# **5) Configuration**

In [ ]:
# `Digital Elevation Models`
demConfigs = {
  "CGIAR": {    # `SRTM Digital Elevation Data Version 4`
    "name": "CGIAR/SRTM90_V4"
  },
  "NASA": {     # `NASA NASADEM Digital Elevation`
    "name": "NASA/NASADEM_HGT/001"
  },
  "USGS": {     # `NASA SRTM Digital Elevation`
    "name": "USGS/SRTMGL1_003"
  },
  "ASTER": {    # AG100: ASTER Global Emissivity Dataset 100-meter V003`
    "name": "NASA/ASTER_GED/AG100_003"
  }
}

# `Sentinel-1 GRD`
s1Config = {
  "name": "COPERNICUS/S1_GRD"
}

bandCombinations = {
  "sum": {
    "expression": "b1 + b2",
    "name": "b1b2S"
  },
  "difference": {
    "expression": "b1 - b2",
    "name": "b1b2D"
  },
  "product": {
    "expression": "b1 * b2",
    "name": "b1b2P"
  },
  "quotient": {
    "expression": "b1 / b2",
    "name": "b1b2Q"
  },
  "ndpi": {
    "expression": "(b1 - b2) / (b1 + b2)",
    "name": "NDPI"
  }
}

In [ ]:
# Parse the appropriate configuration.
extension = os.path.splitext(configFile)[-1]

try:
  with open(configFile, "r") as stream:
    # JSON
    if extension == ".json":
      caseConfigs = json.load(stream)
    # YAML
    elif extension in (".yaml", ".yml"):
      caseConfigs = yaml.safe_load(stream)
    else:
      raise ValueError(f"Unsupported file format `{extension}`. Supported formats: `JSON`, `YAML`")

except FileNotFoundError as e:
  print(f"Error: JSON file not found: {e}")

# GEE assets
demConfig = demConfigs[demProvider]

caseConfig = caseConfigs[caseCode][caseArea]
caseConfig["area_of_interest"] = ee.FeatureCollection(caseConfig["area_of_interest"])

# **6) Data Processing**

Define the projection of interest.

In [ ]:
projection = ee.Projection(projectionCRS).atScale(projectionScale)

Load, filter and process raster collections.

In [ ]:
# `Digital Elevation Elevation`
elevation = ee.Image(demConfig["name"]).unmask()
slope = ee.Terrain.slope(elevation)

# `Sentinel-1 GRD`
preEventRaster = ee.Image("/".join([s1Config["name"], preEventRasterIdentifier]))
postEventRaster = ee.Image("/".join([s1Config["name"], postEventRasterIdentifier]))

preEventRaster = preEventRaster.addBands(**{
  "srcImg": preEventRaster.select("angle").updateMask(preEventRaster.select("angle").mask().gt(0)),
  "overwrite": True,
})

postEventRaster = postEventRaster.addBands(**{
  "srcImg": postEventRaster.select("angle").updateMask(postEventRaster.select("angle").mask().gt(0)),
  "overwrite": True,
})

acquisition = postEventRaster.date()
eventDate = ee.Date(caseConfig["event_date"])

# Apply angular-based radiometric slope correction.
preEventRaster = slope_correction(preEventRaster, elevation)
postEventRaster = slope_correction(postEventRaster, elevation)

# Apply a Refined-Lee speckle speckle noise filter.
preEventRaster = ee.Image(refined_lee(preEventRaster))    \
  .select(["VV", "VH"])                                   \
  .clipToCollection(caseConfig["area_of_interest"])       \
  .reproject(projection)

postEventRaster = ee.Image(refined_lee(postEventRaster))  \
  .select(["VV", "VH"])                                   \
  .clipToCollection(caseConfig["area_of_interest"])       \
  .reproject(projection)

Engineer new raster.

In [ ]:
differenceExpression = bandCombinations["difference"]["expression"]
quotientExpression = bandCombinations["quotient"]["expression"]
ndpiExpression = bandCombinations["ndpi"]["expression"]

differenceName = bandCombinations["difference"]["name"]
quotientName = bandCombinations["quotient"]["name"]

In [ ]:
# `PRE_VVVHD`
prevvprevhDifference = ee.Image().expression(**{
    "expression": differenceExpression,
    "opt_map": {
      "b1": preEventRaster.select(b1),
      "b2": preEventRaster.select(vh)
    }
  })  \
  .rename(differenceName.replace("b1", b1).replace("b2", vh))

# `POST_VVVHD`
postvvpostvhDifference = ee.Image().expression(**{
    "expression": differenceExpression,
    "opt_map": {
      "b1": postEventRaster.select(b1),
      "b2": postEventRaster.select(vh)
    }
  })  \
  .rename(differenceName.replace("b1", b1).replace("b2", vh))

# `VVVVD`
postvvprevvDifference = ee.Image().expression(**{
    "expression": differenceExpression,
    "opt_map": {
      "b1": postEventRaster.select(b1),
      "b2": preEventRaster.select(b1)
    }
  })  \
  .rename(differenceName.replace("b1", b1).replace("b2", b1))

# `VHVHD`
postvhprevhDifference = ee.Image().expression(**{
    "expression": differenceExpression,
    "opt_map": {
      "b1": postEventRaster.select(vh),
      "b2": preEventRaster.select(vh)
    }
  })  \
  .rename(differenceName.replace("b1", vh).replace("b2", vh))

# `VVVHD`
postvvprevhDifference = ee.Image().expression(**{
    "expression": differenceExpression,
    "opt_map": {
      "b1": postEventRaster.select(b1),
      "b2": preEventRaster.select(vh)
    }
  })  \
  .rename(differenceName.replace("b1", b1).replace("b2", vh))

# `PRE_VVVHQ`
prevvprevhQuotient = ee.Image().expression(**{
    "expression": quotientExpression,
    "opt_map": {
      "b1": preEventRaster.select(b1),
      "b2": preEventRaster.select(vh)
    }
  })  \
  .rename(quotientName.replace("b1", b1).replace("b2", vh))

# `POST_VVVHQ`
postvvpostvhQuotient = ee.Image().expression(**{
    "expression": quotientExpression,
    "opt_map": {
      "b1": postEventRaster.select(b1),
      "b2": postEventRaster.select(vh)
    }
  })  \
  .rename(quotientName.replace("b1", b1).replace("b2", vh))

# `VVVVQ`
postvvprevvQuotient = ee.Image().expression(**{
    "expression": quotientExpression,
    "opt_map": {
      "b1": postEventRaster.select(b1),
      "b2": preEventRaster.select(b1)
    }
  })  \
  .rename(quotientName.replace("b1", b1).replace("b2", b1))

# `VHVHQ`
postvhprevhQuotient = ee.Image().expression(**{
    "expression": quotientExpression,
    "opt_map": {
      "b1": postEventRaster.select(vh),
      "b2": preEventRaster.select(vh)
    }
  })  \
  .rename(quotientName.replace("b1", vh).replace("b2", vh))

# `VVVHQ`
postvvprevhQuotient = ee.Image().expression(**{
    "expression": quotientExpression,
    "opt_map": {
      "b1": postEventRaster.select(b1),
      "b2": preEventRaster.select(vh)
    }
  })  \
  .rename(quotientName.replace("b1", b1).replace("b2", vh))

# `PRE_NDPI`
preNDPI = ee.Image().expression(**{
    "expression": ndpiExpression,
    "opt_map": {
      "b1": preEventRaster.select(b1),
      "b2": preEventRaster.select(vh)
    }
  })  \
  .rename("PRE_NDPI")

# `POST_NDPI`
postNDPI = ee.Image().expression(**{
    "expression": ndpiExpression,
    "opt_map": {
      "b1": postEventRaster.select(b1),
      "b2": postEventRaster.select(vh)
    }
  })  \
  .rename("POST_NDPI")

# `NDPID`
NDPIDifference = postNDPI.subtract(preNDPI).rename("NDPID")

# Rename S1-GRD raster bands using a regular expression.
preEventSamplesSource = preEventRaster.regexpRename("V", "PRE_V", False)
postEventSamplesSource = postEventRaster.regexpRename("V", "POST_V", False)

# Add bands to both pre- & post- event rasters.
preEventRaster = preEventRaster.addBands([prevvprevhQuotient])
postEventRaster = postEventRaster.addBands([postvvpostvhQuotient])

# Generate a composite with the essential bands for classification.
raster = preEventSamplesSource.addBands([
  postEventSamplesSource, postvvprevvDifference, postvhprevhDifference,
  postvvprevhDifference, postvvprevvQuotient, postvhprevhQuotient,
  postvvprevhQuotient, preNDPI, postNDPI, NDPIDifference,
])

Raster classification

In [ ]:
# Construct a RF classifier from the decision tree feature collection.
trees = ee.FeatureCollection(classifierIdentifier).aggregate_array("tree")
classifier = ee.Classifier.decisionTreeEnsemble(trees)

# Perform classification.
classified = raster.select(classifierFeatures).classify(classifier)

# **7) Console**

In [ ]:
print("*dates*")
print(f"\t event date: `{eventDate.format('YYYY-MM-DD').getInfo()}`")
print(f"\t acquisition date: `{acquisition.format('YYYY-MM-DD').getInfo()}`")
print(f"\t difference (in days): `{acquisition.difference(eventDate, 'days').getInfo()}`")

*dates*
	 event date: `2018-03-86`
	 acquisition date: `2023-09-250`
	 difference (in days): `1990.0584722222222`


# **8) Export Tasks**

Submit tasks.

In [ ]:
exportTask = ee.batch.Export.image.toAsset(**{
  "description": rasterIdentifier,
  "maxPixels": 1e13,
  "image": classified,
  "scale": projectionScale,
  "crs": projectionCRS,
  "region": caseConfig["area_of_interest"].geometry(),
  "assetId": "/".join([destinationFolder, rasterIdentifier])
})

exportTask.start()

Monitor tasks.

In [ ]:
export_tasks_viewer([exportTask.id])

Task_Id                   Task_State    Task_Type       Task_Attempt  Task_Description                               Queue_Time    Execution_Time    Completion_Time  Error_Message
WPRTRDKCUBIIYWQ5X77EZ5ER  COMPLETED     EXPORT_IMAGE               1  041299_04E8BD_E9C5_050224_060B99_D80F_slope        13.503           925.756            939.259


-End of Notebook-